# Kernel Calculation Test
This notebook is used to test the functionality of the kernel calc methods that (if working) will be rewritten in Python script.

In [13]:
import torch
import resources as rs

### Loading Model Data

In [14]:
model_path = 'ex_models.pt'
state_dict = torch.load(model_path, map_location=torch.device('cpu')) # MODEL

### Unpacking the params

**NOTE:** 
The model was trained on the MNIST digits of 0 1

Unpacking the features params

In [15]:
import torch.nn as nn

# get the weights and biases of the quantized model (for the features layer)
f_weights_quant = state_dict['features.hidden_layer._packed_params._packed_params'][0]
f_bias_quant = state_dict['features.hidden_layer._packed_params._packed_params'][1]

# dequantize the weights and biases
f_weights_float = torch.dequantize(f_weights_quant)
f_bias_float = torch.dequantize(f_bias_quant)
# print the float values of weights and biases
print(f_weights_float)
print(f_bias_float)

tensor([[ 0.0212, -0.0311, -0.0050,  ..., -0.0343, -0.0193,  0.0000],
        [ 0.0112, -0.0050,  0.0149,  ..., -0.0093, -0.0106,  0.0336],
        [-0.0324, -0.0006, -0.0062,  ...,  0.0311,  0.0137, -0.0324],
        ...,
        [-0.0305,  0.0168, -0.0293,  ...,  0.0237, -0.0280, -0.0112],
        [-0.0349,  0.0056, -0.0174,  ..., -0.0093, -0.0031,  0.0212],
        [ 0.0349,  0.0168, -0.0206,  ..., -0.0149,  0.0044, -0.0187]])
Parameter containing:
tensor([ 0.0235, -0.0103, -0.0210,  ..., -0.0244,  0.0028, -0.0313],
       grad_fn=<NotImplemented>)


Unpacking the readout params.

In [16]:
# get the weights and biases of the quantized model (for the readout layer)
r_weights_quant = state_dict['readout._packed_params._packed_params'][0]
r_bias_quant = state_dict['readout._packed_params._packed_params'][1]

# dequantize the weights and bises
r_weights_float = torch.dequantize(r_weights_quant)
r_bias_float = torch.dequantize(r_bias_quant)

print(r_weights_float)
print(r_bias_float)

tensor([[ 0.0219,  0.0389,  0.0024,  ..., -0.0073,  0.0182, -0.0158],
        [-0.0195, -0.0219, -0.0024,  ...,  0.0073, -0.0170,  0.0134]])
Parameter containing:
tensor([0.4623, 0.4824], grad_fn=<NotImplemented>)


### Manually updating the model

In [17]:
model = rs.NN()
model

NN(
  (features): Sequential(
    (hidden_layer): Linear(in_features=784, out_features=2048, bias=True)
    (hidden_activation): ReLU()
  )
  (readout): Linear(in_features=2048, out_features=2, bias=True)
)

In [18]:
params = list(model.parameters())
params

[Parameter containing:
 tensor([[-0.0131, -0.0337, -0.0328,  ...,  0.0347, -0.0070,  0.0193],
         [ 0.0064, -0.0243, -0.0212,  ..., -0.0205, -0.0091,  0.0355],
         [-0.0175, -0.0312, -0.0089,  ..., -0.0182, -0.0322, -0.0102],
         ...,
         [-0.0299,  0.0289,  0.0093,  ..., -0.0277, -0.0058,  0.0048],
         [-0.0191, -0.0097, -0.0204,  ..., -0.0277,  0.0094,  0.0182],
         [ 0.0283,  0.0018,  0.0134,  ...,  0.0321,  0.0254, -0.0120]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0234, -0.0305,  0.0227,  ..., -0.0006, -0.0104, -0.0256],
        requires_grad=True),
 Parameter containing:
 tensor([[ 0.0176, -0.0018, -0.0044,  ..., -0.0051, -0.0119, -0.0023],
         [ 0.0138, -0.0057,  0.0194,  ...,  0.0196,  0.0038,  0.0147]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0073,  0.0175], requires_grad=True)]

In [19]:
params[0].data = f_weights_float
params[1].data = f_bias_float
params[2].data = r_weights_float
params[3].data = r_bias_float

### Loading MNIST dataset

In [20]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
def mnist_dataset(batch_size, train=True, values=list(range(10))):
    # Initializing MNIST data set.
    dataset = datasets.MNIST(root='dataset/', train=train, transform=transforms.ToTensor(), download=True)

    targets_list = dataset.targets.tolist()
    values_index = [i for i in range(len(dataset)) if targets_list[i] in values]

    # Creating a subset of ### MNIST targets.
    subset = torch.utils.data.Subset(dataset, values_index)
    loader = DataLoader(dataset=subset, shuffle=True)

    return loader

In [27]:
MNIST = rs.mnist_dataset(batch_size=0, train=True, values=[0,1])
data, targets = next(iter(MNIST))

Next, in order to perform the CKA calc. we will need to reshape the data into a batch_size X features tenso (12665, 784).

In [28]:
data = torch.squeeze(data, dim=1)
data = data.view(data.size(0), -1)
data.shape

torch.Size([12665, 784])

### CKA Calc.
At this point we can now calculate the CKA for the model state.

In [29]:
model.features(data).shape

torch.Size([12665, 1024])

In [30]:
rs.kernel_calc(targets, model.features(data))

tensor(0.9510, grad_fn=<DivBackward0>)

#TODO using this notebook create a python script that will do this for each of the model states (135 * 512)...